# <b> CRAWLING DATA DETAIL DARI KOPERASI  </b>

proses penarikan data detail koperasi melalui beberapa tahap dikarenakan tidak memiliki endpoint tertentu untuk menampilkan secara keseluruhan dari detail koperasi 
- melakukan penarikan data dari api `https://data.diskopukm.jatimprov.go.id/satu_data/webnya/ajax_listOds`, dari data yang di ambil pada api tersebut, kita menyimpan ID_KOP yang dimana nanti akan digunakan dalam penarikan detail koperasi
- dari ID_KOP yang di dapatkan, kita dapat menarik data dr setiap ID_KOP pada api `https://data.diskopukm.jatimprov.go.id/satu_data/webnya/detailOds/ID_KOP`
- data yang sudah di dapatkan ditampung pada dataframe menggunakan pandas.

> untuk data yang diambil samplenya sebanyak 100 records


In [3]:
import re
import requests
import concurrent.futures
import pandas as pd

def fetch_data(page):
    url = 'https://data.diskopukm.jatimprov.go.id/satu_data/webnya/ajax_listOds'
    payload = {
        'draw': 1,
        'columns[0][data]': '0',
        'columns[0][name]': '',
        'columns[0][searchable]': 'true',
        'columns[0][orderable]': 'true',
        'columns[0][search][value]': '',
        'columns[0][search][regex]': 'false',
        'order[0][column]': '0',
        'order[0][dir]': 'asc',  
        'start': page,
        'length': 10,
        'search[value]': '',
        'search[regex]': 'false',
    }
    response = requests.post(url, data=payload)
    if response.status_code == 200:
        return response.json()['data']
    else:
        print("Failed to fetch data")
        return []

def extract_id_kop(text):
    match = re.search(r'(\d+)', text)
    if match:
        return match.group(1)
    else:
        return None

def fetch_data_detail(base_url, coop_id):
    url = base_url + coop_id
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

def main():
    num_data = 100
    num_requests = num_data // 10 + (num_data % 10 > 0)

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(fetch_data, page) for page in range(num_requests)]
        cooperative_data = []
        for future in concurrent.futures.as_completed(futures):
            cooperative_data.extend(future.result())

    column_titles = ['NO', 'ID KOP', 'KOPERASI', 'ALAMAT', 'NO BADAN HUKUM', 'TANGGAL BADAN HUKUM', 'DESA', 'KELURAHAN', 'AKSI']

    df = pd.DataFrame(cooperative_data, columns=column_titles)
    
    df['ID KOP'] = df['AKSI'].apply(extract_id_kop)
    
    pd.set_option('display.max_colwidth', None)

    # print("Data  Koperasi:")
    # print(df[['ID KOP']].to_string(index=False))  # Hanya menampilkan kolom 'ID KOP'

    base_url = 'https://data.diskopukm.jatimprov.go.id/satu_data/webnya/detailOds/'
    id_kops = df['ID KOP'].tolist()

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(fetch_data_detail, base_url, id_kop) for id_kop in id_kops]
        cooperative_data_detail = []
        for future in concurrent.futures.as_completed(futures):
            cooperative_data_detail.append(future.result())

    df_detail = pd.DataFrame(cooperative_data_detail)

    print("\nData Detail Koperasi:")
    print(df_detail.to_string(index=False))

if __name__ == "__main__":
    main()



Data Detail Koperasi:
         IDKOP                                                   KOPERASI                  NO_BADAN_HUKUM TGL_BADAN_HUKUM                                                      ALAMAT           DESA      KELURAHAN   KECAMATAN       KABUPATEN   PROPINSI KODEPOS NO_TELPON_1 NO_TELPON_2 FAKSIMILI                        EMAIL WEBSITE AKTIF                   KETUA NO_TELP_KETUA           SEKRETARIS NO_TELP_SEKRETARIS                BENDAHARA NO_TELPON_BENDAHARA       BENTUK_KOPERASI JENIS_KOPERASI                         KELOMPOK_KOPERASI                                                            SEKTOR_USAHA UNIT_USAHA         KLASIFIKASI   KESEHATAN THN_IND_KELEMBAGAAN ANGGOTA_PRIA ANGGOTA_WANITA JUMLAH_ANGGOTA MANAJER_PRIA MANAJER_WANITA JUMLAH_MANAJER KARYAWAN_PRIA KARYAWAN_WANITA JUMLAH_KARYAWAN        RAT TAHUN_IND_USAHA MODAL_SENDIRI MODAL_LUAR      ASSET VOLUME_USAHA SISA_HASIL_USAHA  KUK IDPROPINSI IDKABUPATEN IDKECAMATAN IDKELURAHAN IDKOPERASI
3208120100001  K